In [1]:
from utils.config_parse import params
from load_dataset import load_dataset
from initialize_subactivity_assignments import initialize_subactivity_assignments
from embedding.init_embedding_weights import init_embedding_weights
from embedding.optimise_embedding_weights import optimise_embedding_weights
from embedding.feature_extraction import extract_features
#from gauss_mix.compute_gauss_mix import compute_gauss_mix
from utils.load_word_vectors import get_label_word2vec

#from embedding.optimise_embedding_weights import optimise_embedding_weights

#subacts_count == g_K
#vids_frames_gt_label == gt_labels
#latent space transformation Weight: W {Wf, Wa}

# LOAD DATA
activity = "coffee"
path = r"C:/Users/dcsang/Documents/code_translate/DATA/breakfast_actions/formatted_data_new/"
vids_frames_features, vids_frames_gtlabel, vids_frames_count, subacts_count = load_dataset(activity, path)

#RANDOM INITIALISATION OF ACTIVITY LABELS
__rho_prior = params["gmm"]["infOut"]["rho_prior"]
samples = initialize_subactivity_assignments(vids_frames_count, subacts_count, __rho_prior)

###GET VECS/FEATURES FOR LABELS (aka TextFeat in original code)
#TODO do we need number_canonical and number_shuffle? 
#made changes in this structure. in orig code this code is present within fs_prep_embedding_data
labels_features= get_label_word2vec(path, subacts_count)

###EMBEDDING PART
path2embeddingdata_folder = r"C:/Users/dcsang/Documents/code_translate/BREAKFAST_EXP/breakfast_iter/intermediate/INTERMEDIATE_DATA/FINAL_breakfast_iter/coffee/experiment_k_6/v_0_s_50_e_1_nGauss_3word2vec/rand_iter_1/iterative_1"
W = init_embedding_weights() #this is  initParams of original code
W = optimise_embedding_weights(path2embeddingdata_folder, vids_frames_features, 
                               vids_frames_count, subacts_count, samples[0]["z"],W, labels_features)
#TODO: samples has 0-padding for vids with frames less than max_frames_count,
#and iteration_seglabels is basically creating a DS with variable row sizes and removing this padding
#i am going ahead with samples, will change later if necessary

###FEATURE EXTRACTION
X_probs = extract_features(labels_features, vids_frames_features, W)

Dataset Loaded Successfully
Uniform initialisation of subactivity labels complete
Started optimising W
Starting Optimisation
iter 1/10 (0.0% done): raw_cost: 17948446.681779 	 reg_cost: 0.000000 	 total_cost: 17948446.681779

iter 2/10 (10.0% done): raw_cost: 17946298.325956 	 reg_cost: 0.000000 	 total_cost: 17946298.325956

iter 3/10 (20.0% done): raw_cost: 17944051.697430 	 reg_cost: 0.000000 	 total_cost: 17944051.697430

iter 4/10 (30.0% done): raw_cost: 17942959.979215 	 reg_cost: 0.000000 	 total_cost: 17942959.979215

iter 5/10 (40.0% done): raw_cost: 17944726.379082 	 reg_cost: 0.000000 	 total_cost: 17944726.379082

iter 6/10 (50.0% done): raw_cost: 17942232.463406 	 reg_cost: 0.000000 	 total_cost: 17942232.463406

iter 7/10 (60.0% done): raw_cost: 17943028.390031 	 reg_cost: 0.000000 	 total_cost: 17943028.390031

iter 8/10 (70.0% done): raw_cost: 17945835.747965 	 reg_cost: 0.000000 	 total_cost: 17945835.747965

iter 9/10 (80.0% done): raw_cost: 17947581.114380 	 reg_cost

C:\Users\dcsang\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dcsang\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [2]:
X_probs.shape

AttributeError: 'list' object has no attribute 'shape'

In [3]:
vids_frames_label,vids_frames_count, X_probs

167

In [5]:
vids_frames_label = samples[0]["z"]

In [10]:
num_clusters = 3
vids_count = len(vids_frames_count)
frames_count = sum(vids_frames_count)

In [15]:
frames_label_concat = np.zeros(frames_count)

In [20]:
idx =0
for vi in range(vids_count):
    fc = vids_frames_count[vi]
    temp = vids_frames_label[vi, :fc]
    frames_label_concat[idx:idx+fc] = temp
    idx+=fc

In [ ]:
X_probs_concat = np.zeros([frames_count,])

In [14]:
import numpy as np

In [16]:
vids_frames_label.shape

(167, 1204)

In [21]:
idx

97290

In [23]:
frames_label_concat

array([1., 1., 1., ..., 6., 6., 6.])

In [31]:
blah = []
for vi in range(vids_count):
    fc = vids_frames_count[vi]
    temp = vids_frames_label[vi, :fc]
    blah.extend(temp)

In [32]:
len(blah)

97290

In [33]:
np.unique(blah)

array([1, 2, 3, 4, 5, 6], dtype=uint32)

In [34]:
def concat_probs(X_probs, vids_count):
    output =[]
    for vi in range(vids_count):
        output.extend(X_probs[vi])
    return output

In [42]:
X_probs_concat = concat_probs(X_probs, vids_count)

In [40]:
X_probs_concat = np.array(X_probs_concat)

In [43]:
len(X_probs_concat)

1002

In [63]:
len(X_probs[0].tolist())

6

In [58]:
blah = X_probs[0].tolist()

In [59]:
len(blah)

6

In [52]:
blah.extend(X_probs[1])

In [56]:
len(blah)

6

In [67]:
bobo = np.zeros([subacts_count, frames_count])
idx = 0
for vi in range(vids_count):
    fc = vids_frames_count[vi]
    bobo[: ,idx:idx+fc] = X_probs[vi]
    idx += fc

In [68]:
bobo.shape

(6, 97290)